# Notebook 1 - Explorando o GSM Gemynocystis sp. GBB08 criado com CarveMe

## Apresentação do Notebook

Neste notebook consta as intervenções no modelo da Geminocystis GBBB08 realizado pelo carveme.  

In [1]:
import cobra

In [ ]:
import pandas as pd

## 1. Carregando modelo 

In [2]:
model = cobra.io.read_sbml_model('/mnt/d/Document/Mestrado/Dissertação/dissertacao-ufma/Material_suplementar/carveme/model_sbml/iGbb08.xml')


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## 2. Inspecionando o modelo 

In [3]:
model

Name,iGbb08
Memory address,0x07fe17eae0f40
Number of metabolites,1127
Number of reactions,1509
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space, thylakoid"


In [4]:
model_list = [model]
for model in model_list:
    num_reactions = len(model.reactions)
    num_metabolites = len(model.metabolites)
    num_genes = len(model.genes)

    print("Model information")
    print("Model ID is: {}".format(model.id))
    print("Number of reactions: {}".format(num_reactions))
    print("Number of metabolites: {}".format(num_metabolites))
    print("Number of genes: {}".format(num_genes))
    print("Model compartments:")
    for k, v in model.compartments.items():
        print(k + '\t' + v)
    print('\n################################\n')

Model information
Model ID is: iGbb08
Number of reactions: 1509
Number of metabolites: 1127
Number of genes: 758
Model compartments:
C_c	cytosol
C_p	periplasm
C_e	extracellular space
C_u	thylakoid

################################



# Inspecionando a Função Objetivo (FBA) e os Fluxos de Troca (FVA)

# 3. Inspecionando FBA

As inspeções serão para os três metabólitos de interesse e primeiramente serão realizadas apenas a mudança da função objetivo.
Contudo, deve-se avaliar se há formação de biomassa! 

In [5]:
# Quais são os metabólitos que estão retornando para mim?
## Uptake - o que a simulação consegue captar do meio 
## Secretion - Simula o que a célula iria exportar!

model.summary() #mmol/gDW/h



Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0001104,0,0.00%
cit_e,EX_cit_e,0.0003311,6,0.22%
cl_e,EX_cl_e,0.0001104,0,0.00%
co2_e,EX_co2_e,0.9028,1,99.78%
cobalt2_e,EX_cobalt2_e,2.121E-06,0,0.00%
cu2_e,EX_cu2_e,1.503E-05,0,0.00%
fe2_e,EX_fe2_e,0.0001424,0,0.00%
fe3_e,EX_fe3_e,0.0001656,0,0.00%
h2o_e,EX_h2o_e,0.3049,0,0.00%
k_e,EX_k_e,0.004139,0,0.00%


In [6]:
solution = model.optimize()
print(solution)


<Solution 0.022 at 0x7fe139f9be20>


In [7]:
model.medium  #meio de cultivo

{'EX_h2o_e': 10.0,
 'EX_hco3_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cit_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_co2_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_no3_e': 10.0,
 'EX_photon_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_cu2_e': 10.0}

### 3.1 - Principais reações de consumo e produção de energia

In [ ]:
model.metabolites.atp_c.summary()

In [ ]:
model.reactions.Growth.summary() #teste de reação de Biomassa

## 4 - FVA, pFBA com Restrições ao Fenótipo do Modelo

In [8]:
from cobra.flux_analysis import flux_variability_analysis

In [9]:
flux_variability_analysis(model, model.reactions[1502])


,minimum,maximum
Growth,0.021897,0.021897


In [10]:
flux_variability_analysis(model, model.reactions[406])

,minimum,maximum
DMATT,0.002123,0.002123


In [11]:
flux_variability_analysis(model, model.reactions[625])

,minimum,maximum
GRTT,0.002123,0.002123


In [12]:
flux_variability_analysis(model, model.reactions[416])

,minimum,maximum
DPPS,0.002121,0.002121


In [13]:
model.optimize()
model.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0001104,[0.0001049; 0.0001104],0,0.00%
cit_e,EX_cit_e,0.0003311,[0; 0.0003311],6,0.22%
cl_e,EX_cl_e,0.0001104,[0.0001049; 0.0001104],0,0.00%
co2_e,EX_co2_e,0.9028,[0.65; 0.9389],1,99.78%
cobalt2_e,EX_cobalt2_e,2.121E-06,[2.015E-06; 2.121E-06],0,0.00%
cu2_e,EX_cu2_e,1.503E-05,[1.428E-05; 1.503E-05],0,0.00%
fe2_e,EX_fe2_e,0.0001424,[0.0001353; 0.001455],0,0.00%
fe3_e,EX_fe3_e,0.0001656,[0; 0.0001656],0,0.00%
h2o_e,EX_h2o_e,0.3049,[0.04921; 0.3552],0,0.00%
k_e,EX_k_e,0.004139,[0.003932; 0.004139],0,0.00%


## 4.1. FVA para a Via MEP

In [14]:
FVA = cobra.flux_analysis.flux_variability_analysis(model, processes = 8, loopless = True, pfba_factor = 1.1)

In [15]:
MEP_reactions = ['DXPS', 'DXPRIi', 'MEPCT', 'CDPMEK', 'MECDPS',
                'MECDPDH_syn', 'IPDPS', 'DMATT',
                'GRTT', 'DPPS', 'Growth', 'IPDPS_syn']

In [16]:
MEP_reacations_FVA = FVA.loc[MEP_reactions]
MEP_reacations_FVA

,minimum,maximum
DXPS,0.023353,0.023353
DXPRIi,0.023343,0.023343
MEPCT,0.023343,0.023343
CDPMEK,0.023343,0.023343
MECDPS,0.023343,0.023343
MECDPDH_syn,0.023343,0.023343
IPDPS,0.000000,0.000000
DMATT,0.002123,0.002123
GRTT,0.002123,0.002123
DPPS,0.002121,0.002121


## 4.1.2. Alteração da Solução Objetivo (1° Restrição de Fenótipo)

In [17]:
solutions = {}
objectives = ['Growth','DMATT', 'GRTT', 'DPPS']

for objective in objectives:
    with model:
        model.objective = objective
        print(model.objective.expression)
        solution = model.optimize()
        solutions[objective] = solution.objective_value
    print(model.objective.expression)

1.0*Growth - 1.0*Growth_reverse_699ae
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DMATT - 1.0*DMATT_reverse_3a731
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*GRTT - 1.0*GRTT_reverse_f3afe
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DPPS - 1.0*DPPS_reverse_d6ed6
1.0*Growth - 1.0*Growth_reverse_699ae


In [18]:
solutions #valor de FBA com a mudança do objetivo para as reações de terpenóides

{'Growth': 0.02189715598358892,
 'DMATT': 0.019320843091334593,
 'GRTT': 0.019320843091334593,
 'DPPS': 0.011138446767975122}

## 4.1.3. Alteração da Solução Objetivo e visuazliação de pFBA

In [19]:
results_fluxes_growth = {}
results_list = {}
reactions = ['DMATT','GRTT', 'DPPS']

for reaction in reactions:
    with model:
        model.objective = reaction
        print(model.objective.expression)
        fba_solution = model.optimize()
        pfba_solution = cobra.flux_analysis.parsimonious.pfba(model, fraction_of_optimum= 1.0)
        results_list[reaction] = pfba_solution.objective_value
        print(pfba_solution.fluxes["Growth"])
        results_fluxes_growth[reaction] = pfba_solution.fluxes["Growth"]
    print(model.objective.expression)

1.0*DMATT - 1.0*DMATT_reverse_3a731
6.67242810957958e-19
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*GRTT - 1.0*GRTT_reverse_f3afe
-1.8174125185489555e-20
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DPPS - 1.0*DPPS_reverse_d6ed6
-1.2961106951006021e-30
1.0*Growth - 1.0*Growth_reverse_699ae


In [20]:
results_list # Solução de pFBA

{'DMATT': 63.09718969555151,
 'GRTT': 63.09718969555314,
 'DPPS': 63.81923088462504}

In [21]:
results_fluxes_growth # O valor de pFBA para o fluxo de Biomassa, iterando para cada uma dessas reações. -> pfba_solution.fluxes["Growth"]

{'DMATT': 6.67242810957958e-19,
 'GRTT': -1.8174125185489555e-20,
 'DPPS': -1.2961106951006021e-30}

## 4.1.3. Alteração da Solução Objetivo e visuazliação de pFBA (2) 

#### solution of optimum = 0.2

In [29]:
results_fluxes_growth_fo2 = {}
results_list_fo2 = {}
reactions = ['Growth','DMATT','GRTT', 'DPPS']

for reaction in reactions:
    with model:
        model.objective = reaction
        print(model.objective.expression)
        fba_solution = model.optimize()
        pfba_solution = cobra.flux_analysis.parsimonious.pfba(model, fraction_of_optimum= 0.2)
        results_list_fo2[reaction] = pfba_solution.objective_value
        print(pfba_solution.fluxes["Growth"])
        results_fluxes_growth_fo2[reaction] = pfba_solution.fluxes["Growth"]
    print(model.objective.expression)

1.0*Growth - 1.0*Growth_reverse_699ae
0.004379431196717786
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DMATT - 1.0*DMATT_reverse_3a731
0.0
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*GRTT - 1.0*GRTT_reverse_f3afe
0.0
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DPPS - 1.0*DPPS_reverse_d6ed6
0.0
1.0*Growth - 1.0*Growth_reverse_699ae


In [30]:
results_list_fo2

{'Growth': 12.623092492934301,
 'DMATT': 12.554683840749437,
 'GRTT': 12.554683840749437,
 'DPPS': 12.685185115734908}

In [31]:
results_fluxes_growth_fo2

{'Growth': 0.004379431196717786, 'DMATT': 0.0, 'GRTT': 0.0, 'DPPS': 0.0}

In [27]:
### Exemplo

solutions_02 = {}
objectives = ['Growth','DMATT', 'GRTT', 'DPPS']

for objective in objectives:
    with model:
        model.objective = objective
        print(model.objective.expression)
        solution = model.optimize()
        solutions_02[objective] = solution.objective_value
    print(model.objective.expression)

1.0*Growth - 1.0*Growth_reverse_699ae
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DMATT - 1.0*DMATT_reverse_3a731
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*GRTT - 1.0*GRTT_reverse_f3afe
1.0*Growth - 1.0*Growth_reverse_699ae
1.0*DPPS - 1.0*DPPS_reverse_d6ed6
1.0*Growth - 1.0*Growth_reverse_699ae


In [28]:
solutions_02

{'Growth': 0.021897155983588604,
 'DMATT': 0.019320843091334906,
 'GRTT': 0.019320843091334906,
 'DPPS': 0.011138446767975265}

In [32]:
cobra.flux_analysis.variability.flux_variability_analysis(model, reaction_list=['Growth','DMATT', 'GRTT', 'DPPS'],loopless = True, pfba_factor = 1.1)

,minimum,maximum
Growth,0.021897,0.021897
DMATT,0.002123,0.002123
GRTT,0.002123,0.002123
DPPS,0.002121,0.002121


### Observação

Nessa etapa foi observado a presença da 'IPDPS' e da 'IDPS_syn' -> Ambas tiveram anotação diferente neste modelo. Sendo que, ambas apresentam o Fluxo nulo. Sendo assim, foi feita a remoção dessas e novamente a simulação tanto para o FVA e tanto para o pFBA

###### Deleção das reações

In [ ]:
model.remove_reactions(['IPDPS_syn', 'DMPPS_syn'])

## 4.2. FVA para Via MEP modificada

In [ ]:
FVA_rm = cobra.flux_analysis.flux_variability_analysis(model, processes = 8, loopless = True, pfba_factor = 1.1)

In [ ]:
MEP_reactions_rm = ['DXPS', 'DXPRIi', 'MEPCT', 'CDPMEK', 'MECDPS',
                'MECDPDH_syn', 'IPDPS', 'DMATT',
                'GRTT', 'DPPS', 'Growth']

In [ ]:
MEP_reacations_FVA_rm = FVA.loc[MEP_reactions]
MEP_reacations_FVA_rm

## 4.2.2. Alteração da Solução Objetivo (1° Restrição de Fenótipo)

In [ ]:
solutions_1 = {}
objectives = ['Growth','DMATT', 'GRTT', 'DPPS' ]

for objective in objectives:
    with model:
        model.objective = objective
        print(model.objective.expression)
        solution = model.optimize()
        solutions_1[objective] = solution.objective_value
    print(model.objective.expression)

In [ ]:
solutions_1 #FBA

In [ ]:
results_fluxes_growth_1 = {}
results_list_1 = {}
reactions = ['DMATT','GRTT', 'DPPS']

for reaction in reactions:
    with model:
        model.objective = reaction
        print(model.objective.expression)
        fba_solution = model.optimize()
        pfba_solution = cobra.flux_analysis.parsimonious.pfba(model, fraction_of_optimum= 0.2)
        results_list_1[reaction] = pfba_solution.objective_value
        print(pfba_solution.fluxes["Growth"])
        results_fluxes_growth_1[reaction] = pfba_solution.fluxes["Growth"]
    print(model.objective.expression)

In [ ]:
results_list_1

In [ ]:
results_fluxes_growth_1

## 4.2.3. Alteração da Solução Objetivo e visuazliação de pFBA

In [ ]:
results_list_1 = {}
reactions = ['DMATT','GRTT', 'DPPS']

for reaction in reactions:
    with model:
        model.objective = reaction
        print(model.objective.expression)
        fba_solution = model.optimize()
        pfba_solution = cobra.flux_analysis.pfba(model, fraction_of_optimum= 1.0)
        results_list_1[reaction] = fba_solution.objective_value
    print(model.objective.expression)

In [ ]:
results_list_1

In [ ]:
results_list_1

In [ ]:
### 4.3. Demonstração da alteração da Solução Objetivo com o pFBA para a Reação de Biomassa

In [ ]:
Lista_pfba_growth = {}
reactions_objective = ['DMATT','GRTT','DPPS']

for reaction in reactions_objective:
    with model:
        reactions_objective = cobra.flux_analysis.pfba(model)
        Lista_pfba_growth[reaction] = pfba_solution.fluxes['Growth']
        


In [ ]:
Lista_pfba_growth

## Mudança na Solução Objetivo para antes da retirada de IDPS_syn

In [ ]:
solutions_pfba = {}
objectives_pfba = ['Growth','DMATT','GRTT','DPPS']

for i_pfba in objectives_pfba:
    objectives_pfba = []
    cobra.flux_analysis.parsimonious.pfba
    

### Inspecionando a Visualização em Gráfico

### Carregando as Bibliotecas

In [ ]:

import escher
from escher import Builder
import cobra
from time import sleep
escher.rc['never_ask_before_quit'] = True

In [ ]:
builder = Builder(model_json='/mnt/d/Document/Mestrado/Dissertação/Resultados/1_Reconstrução_Metabólica/1_carveme/Jupyter_Notebook_iGBBB/igbb08.json')
builder